<a href="https://colab.research.google.com/github/rahiakela/deep-reinforcement-learning-hands-on/blob/chapter-5-tabular-learning-and-the-bellman-equation/1_value_iteration_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The value iteration method

In the simplistic example to calculate the values of the states and actions, we exploited the structure of the environment: we had no loops in transitions, so we could start from terminal states, calculate their values, and then proceed to the central state. However, just one loop in the environment builds an obstacle in our approach. 

Let's consider such an environment with two states:

<img src='https://github.com/rahiakela/img-repo/blob/master/deep-reinforcement-learning-hands-on/transition.png?raw=1' width='800'/>

We start from state s1, and the only action we can take leads us to state s2. We get the reward r = 1, and the only transition from s2 is an action, which brings us back to s1. So, the life of our agent is an infinite sequence of states `[𝑠1, 𝑠2, 𝑠1, 𝑠2, 𝑠1, 𝑠2, 𝑠1, 𝑠2,…]`.

To deal with this infinity loop, we can use a discount factor: `𝛾 = 0.9` . Now, the question is, what are the values for both the states? The answer is not very complicated, in fact. Every transition from s1 to s2 gives us a reward of 1 and every back transition gives us 2. So, our sequence of rewards will be `[1, 2, 1, 2, 1, 2, 1, 2, ….]`. As there is only one action available in every state, our agent has no choice, so we can omit the max operation in formulas (there is only one alternative).

The value for every state will be equal to the infinite sum:

$$𝑉(𝑠_1) = 1 + 𝛾 (2 + 𝛾(1 + 𝛾(2 + ⋯ ))) = \sum_{i=0}^{\infty}1𝛾^{2𝑖} + 2𝛾^{2𝑖+1}$$

$$𝑉(𝑠_2) = 2 + 𝛾 (1 + 𝛾(2 + 𝛾(1 + ⋯ ))) = \sum_{i=0}^{\infty}2𝛾^{2𝑖} + 1𝛾^{2𝑖+1}$$

**Strictly speaking, we can't calculate the exact values for our states, but with 𝛾𝛾 = 0.9 , the contribution of every transition quickly decreases over time.**

For example, after 10 steps, `𝛾10 = 0.910 = 0.349` , but after 100 steps, it becomes just `0.0000266`. Due to this, we can stop after 50 iterations and still get quite a precise estimation.

In [1]:
sum([0.9 ** (2 * i) + 2 * (0.9 ** (2 * i + 1)) for i in range(50)])

14.736450674121663

In [2]:
sum([2 * (0.9 ** (2 * i)) + 0.9 ** (2 * i + 1) for i in range(50)])

15.262752483911719

The preceding example can be used to get the gist of a more general procedure
called the **value iteration algorithm**. This allows us to numerically calculate the values of the states and values of the actions of **Markov decision processes (MDPs)** with known transition probabilities and rewards.

The procedure (for values of the states) includes the following steps:

1. Initialize the values of all states, $V_i$, to some initial value (usually zero)
2. For every state, $s$, in the MDP, perform the Bellman update:
$$𝑉_s ← max_a \sum_{s'}P_{a,s→s'}(r_{r,s} + \gamma V_{s'})$$

3. Repeat step 2 for some large number of steps or until changes become too
small

In the case of action values (that is, Q), only minor modifications to the preceding procedure are required:

1. Initialize every $Q_{s,a}$ to zero
2. For every state, $s$, and action, $a$, in this state, perform this update:
$$Q_{s,a} ← \sum_{s'}P_{a,s→s'}(r_{r,s} + \gamma max_{a'} Q_{s',a'})$$
3. Repeat step 2

Okay, so that's the theory. In practice, this method has several obvious limitations.

1. First of all, our state space should be discrete and small enough to perform multiple iterations over all states. This is not an issue for FrozenLake-4x4 and even for FrozenLake-8x8 (it exists in Gym as a more challenging version), but for CartPole, it's not totally clear what to do.

2. The second practical problem arises from the fact that we rarely know the
transition probability for the actions and rewards matrix. Remember the interface provided by Gym to the agent's writer: we observe the state, decide on an action, and only then do we get the next observation and reward for the transition.

What we do have is just the history from the agent's interaction with the
environment. However, in Bellman's update, we need both a reward for every
transition and the probability of this transition. So, the obvious answer to this issue is to use our agent's experience as an estimation for both unknowns. Rewards could be used as they are. We just need to remember what reward we got on the transition from $s_0$ to $s_1$ using action a, but to estimate probabilities, we need to maintain counters for every tuple $(s_0, s_1, a)$ and normalize them.

## Value iteration in practice

The central data structures in this example are as follows:

